# Geospatial maps

In [13]:
# HIDDEN
import folium
from folium import Map, TileLayer
import leafmap
import pandas as pd

from tools import get_all_files
from PARAMETERS import DATA_ROOT
from tools import get_range_tif_request

In [11]:
# HIDDEN
# Set the endpoint for the TiTiler server
titiler_endpoint = "http://170.64.253.130:8000" 

In [14]:
# Get all output files
files = get_all_files(DATA_ROOT)

In [26]:
maps = files.query('catetory.str.contains("map") and base_ext == ".tiff" and base_name.str.contains("color") and year_types == "single_year"').reset_index(drop=True)

In [49]:
maps

,year,year_types,catetory,base_name,base_ext,path
0,2010,single_year,ammap,ammap_asparagopsis_taxiformis_color4band,.tiff,/home/jinzhu/docker_data/LUTO_DATA/2023_12_12_...
1,2010,single_year,ammap,ammap_ecological_grazing_color4band,.tiff,/home/jinzhu/docker_data/LUTO_DATA/2023_12_12_...
2,2010,single_year,ammap,ammap_precision_agriculture_color4band,.tiff,/home/jinzhu/docker_data/LUTO_DATA/2023_12_12_...
3,2010,single_year,lmmap,lmmap_color4band,.tiff,/home/jinzhu/docker_data/LUTO_DATA/2023_12_12_...
4,2010,single_year,lumap,lumap_color4band,.tiff,/home/jinzhu/docker_data/LUTO_DATA/2023_12_12_...
...,...,...,...,...,...,...
1360,2050,single_year,lumap_separate_Agricultural Land-use,Agricultural Land-use_25_Winter cereals_color4...,.tiff,/home/jinzhu/docker_data/LUTO_DATA/2023_12_12_...
1361,2050,single_year,lumap_separate_Agricultural Land-use,Agricultural Land-use_26_Winter legumes_color4...,.tiff,/home/jinzhu/docker_data/LUTO_DATA/2023_12_12_...
1362,2050,single_year,lumap_separate_Agricultural Land-use,Agricultural Land-use_27_Winter oilseeds_color...,.tiff,/home/jinzhu/docker_data/LUTO_DATA/2023_12_12_...
1363,2050,single_year,lumap_separate_Non-Agricultural Land-use,Non-Agricultural Land-use_00_Environmental Pla...,.tiff,/home/jinzhu/docker_data/LUTO_DATA/2023_12_12_...


In [31]:
category2name = {'lumap': 'Agricultural Land-use all category',
                'lumap_separate_Agricultural Land-use': 'Agricultural Land-use single category',
                'ammap': 'Agriculture Management',
                'lumap_separate_Non-Agricultural Land-use': 'Non-Agricultural Land-use ',
                'lmmap': 'Dry and Irrigated Land-use'}

In [48]:
base2name = {'Agricultural Land-use_00_Apples_color4band': 'Apples',
            'Agricultural Land-use_01_Beef - modified land_color4band': 'Beef - modified land',
            'Agricultural Land-use_02_Beef - natural land_color4band': 'Beef - natural land',
            'Agricultural Land-use_03_Citrus_color4band': 'Citrus',
            'Agricultural Land-use_04_Cotton_color4band': 'Cotton',
            'Agricultural Land-use_05_Dairy - modified land_color4band': 'Dairy - modified land',
            'Agricultural Land-use_06_Dairy - natural land_color4band': 'Dairy - natural land',
            'Agricultural Land-use_07_Grapes_color4band': 'Grapes',
            'Agricultural Land-use_08_Hay_color4band': 'Hay',
            'Agricultural Land-use_09_Nuts_color4band': 'Nuts',
            'Agricultural Land-use_10_Other non-cereal crops_color4band': 'Other non-cereal crops',
            'Agricultural Land-use_11_Pears_color4band': 'Pears',
            'Agricultural Land-use_12_Plantation fruit_color4band': 'Plantation fruit',
            'Agricultural Land-use_13_Rice_color4band': 'Rice',
            'Agricultural Land-use_14_Sheep - modified land_color4band': 'Sheep - modified land',
            'Agricultural Land-use_15_Sheep - natural land_color4band': 'Sheep - natural land',
            'Agricultural Land-use_16_Stone fruit_color4band': 'Stone fruit',
            'Agricultural Land-use_17_Sugar_color4band': 'Sugar',
            'Agricultural Land-use_18_Summer cereals_color4band': 'Summer cereals',
            'Agricultural Land-use_19_Summer legumes_color4band': 'Summer legumes',
            'Agricultural Land-use_20_Summer oilseeds_color4band': 'Summer oilseeds',
            'Agricultural Land-use_21_Tropical stone fruit_color4band': 'Tropical stone fruit',
            'Agricultural Land-use_22_Unallocated - modified land_color4band': 'Unallocated - modified land',
            'Agricultural Land-use_23_Unallocated - natural land_color4band': 'Unallocated - natural land',
            'Agricultural Land-use_24_Vegetables_color4band': 'Vegetables',
            'Agricultural Land-use_25_Winter cereals_color4band': 'Winter cereals',
            'Agricultural Land-use_26_Winter legumes_color4band': 'Winter legumes',
            'Agricultural Land-use_27_Winter oilseeds_color4band': 'Winter oilseeds',
            'Non-Agricultural Land-use_00_Environmental Plantings_color4band': 'Environmental Plantings',
            'lumap_color4band': 'Land-use Map',
            'lmmap_color4band': 'Dry Irriagted Land-use Map'}

## Land use map

In [ ]:
# HIDDEN

# Get the TIF files from a RANGE_HTTP server
tif_url = 'http://170.64.253.130:8080/LUTO_DATA/CWC15_2023_11_23__07_48_38_hard_mincost_RF5_P1e5_2010-2050_timeseries_-265Mt/out_2030/lumap__color4band.tiff'

# Get tile data from titiler
tile_json_analytic = get_range_tif_request(titiler_endpoint, "cog/tilejson.json", tif_url)
tileset = tile_json_analytic["tiles"][0]

In [ ]:
# HIDDEN

# get the metadata from the legend of land use classes
color_csvs = pd.read_csv('tools/color_map.csv')
legend_dict = dict(color_csvs[['lu_desc','color_HEX']].values)

In [ ]:
# HIDDEN
map = leafmap.Map(center=(-27, 134),
                  zoom=4,
                  draw_control=False,
                  measure_control=False,
                #   fullscreen_control=False,
                  attribution_control=True)

map.add_tile_layer(
    url='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    name="ESRI Satellite",
    attribution="Google",
)

map.add_tile_layer(
    tileset,
    layers="LUTO",
    name="LUTO",
    format="image/png",
    transparent=True,
    attribution='LUTO'
)

map.add_legend(title="LUTO Classification", legend_dict=legend_dict)
map

Map(center=[-27, 134], controls=(AttributionControl(options=['position', 'prefix'], position='bottomright'), Z…

In [ ]:
# HIDDEN
m = Map(
    location=(-27, 134),
    zoom_start=4,
    tiles="Stamen Terrain",
    attr="LUTO 2.0",
)

base_layer = TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = False,
        control = True,
       ).add_to(m)

tile_layer = TileLayer(
    tiles= tileset,
    attr="LUTO 2.0",
)

tile_layer.add_to(m)

In [ ]:
# HIDE CODE
from branca.element import Figure
fig = Figure(width=800, height=600)
fig.add_child(m)